# VMIFGSM/Onepixel

Run VMIFGSM and onepixel for multiple models and epsilons, save outputs, and plot adversarial accuracy.

In [ ]:
import os
import re
import subprocess
import sys
from pathlib import Path

import matplotlib.pyplot as plt

EPSILONS = [0, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3]
PIXELS = [1, 2, 3, 4, 5]
VMIFGSM_MODELS = ["resnet34"]
BLACKMODELS = ["resnet18", "densenet121"]
ONEPIXEL_MODELS = ["resnet18", "densenet121"]
VMIFGSM_DATASET = "cifar10"
ONEPIXEL_DATASET = "cifar10"
MAX_IMAGES = 50
VMIFGSM_SCRIPT = str(Path("vmifgsm.py").resolve())
OP_SCRIPT = str(Path("one-pixel.py").resolve())
OUTPUT_ROOT = Path("output")

def alpha_for_epsilon(epsilon: float) -> float:
    return epsilon * (0.007 / 0.031)


In [ ]:
def run_vmifgsm(model: str, epsilon: float, blackmodel: str) -> tuple[float, float]:
    eps_str = str(epsilon)
    alpha = alpha_for_epsilon(epsilon)
    save_dir = OUTPUT_ROOT / "vmifgsm" / f"{VMIFGSM_DATASET}_{model}_{eps_str}_to_{blackmodel}"
    cmd = [
        sys.executable,
        VMIFGSM_SCRIPT,
        "--model",
        model,
        "--dataset",
        VMIFGSM_DATASET,
        "--epsilon",
        eps_str,
        "--alpha",
        str(alpha),
        "--blackbox-model",
        blackmodel,
        "--max-images",
        str(MAX_IMAGES),
        "--save-dir",
        str(save_dir),
    ]
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    wb_match = re.search(r"VMIFGSM acc: ([0-9.]+)", result.stdout)
    bb_match = re.search(r"Black-box acc: ([0-9.]+)", result.stdout)
    if not wb_match:
        raise RuntimeError(f"Could not parse VMIFGSM acc for {model} eps={epsilon}.")
    if not bb_match:
        raise RuntimeError(f"Could not parse black-box acc for {blackmodel} eps={epsilon}.")
    return float(wb_match.group(1)), float(bb_match.group(1))


In [ ]:
def run_onepixel(model: str, pixels: int) -> float:
    pixels_str = str(pixels)
    save_dir = OUTPUT_ROOT / "onepixel" / f"{ONEPIXEL_DATASET}_{model}_p{pixels_str}"
    cmd = [
        sys.executable,
        OP_SCRIPT,
        "--model",
        model,
        "--dataset",
        ONEPIXEL_DATASET,
        "--pixels",
        pixels_str,
        "--max-images",
        str(MAX_IMAGES),
        "--save-dir",
        str(save_dir),
    ]
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    match = re.search(r"Adv acc: ([0-9.]+)", result.stdout)
    if not match:
        raise RuntimeError(f"Could not parse Adv acc for {model} pixels={pixels}.")
    return float(match.group(1))


In [ ]:
# VMIFGSM sweep
accuracies = {model: {black: [] for black in BLACKMODELS} for model in VMIFGSM_MODELS}
transfer_accuracies = {model: {black: [] for black in BLACKMODELS} for model in VMIFGSM_MODELS}

for model in VMIFGSM_MODELS:
    for blackmodel in BLACKMODELS:
        for epsilon in EPSILONS:
            wb_acc, bb_acc = run_vmifgsm(model, epsilon, blackmodel)
            accuracies[model][blackmodel].append(wb_acc)
            transfer_accuracies[model][blackmodel].append(bb_acc)

accuracies, transfer_accuracies


In [ ]:
# Save VMIFGSM CSV
import csv

vmifgsm_csv = Path("output") / "vmifgsm" / "vmifgsm_accuracy.csv"
vmifgsm_csv.parent.mkdir(parents=True, exist_ok=True)

with vmifgsm_csv.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["model", "blackmodel", "epsilon", "whitebox_acc", "blackbox_acc"])
    for model in VMIFGSM_MODELS:
        for blackmodel in BLACKMODELS:
            for epsilon, wb_acc, bb_acc in zip(
                EPSILONS,
                accuracies[model][blackmodel],
                transfer_accuracies[model][blackmodel],
            ):
                writer.writerow([model, blackmodel, epsilon, wb_acc, bb_acc])

vmifgsm_csv


In [ ]:
# Plot VMIFGSM (white-box)
plt.figure(figsize=(8, 5))
for model in VMIFGSM_MODELS:
    label = f"{model}"
    plt.plot(EPSILONS, accuracies[model]["resnet18"], marker="o", label=label)

plt.xlabel("epsilon")
plt.ylabel("accuracy")
plt.title("VMIFGSM accuracy vs. epsilon")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Plot VMIFGSM (black-box transfer)
plt.figure(figsize=(8, 5))
plt.plot(EPSILONS, accuracies["resnet34"]["resnet18"], marker="o", label=label, color='green')
for model in VMIFGSM_MODELS:
    for blackmodel in BLACKMODELS:
        label = f"{model}->{blackmodel}"
        plt.plot(EPSILONS, transfer_accuracies[model][blackmodel], marker="o", label=label)

plt.xlabel("epsilon")
plt.ylabel("accuracy")
plt.title("VMIFGSM black-box accuracy vs. epsilon")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# OnePixel sweep
onepixel_accuracies = {model: [] for model in ONEPIXEL_MODELS}

for model in ONEPIXEL_MODELS:
    for pixels in PIXELS:
        acc = run_onepixel(model, pixels)
        onepixel_accuracies[model].append(acc)

onepixel_accuracies


In [ ]:
# Save OnePixel CSV
import csv

onepixel_csv = Path("output") / "onepixel" / "onepixel_accuracy.csv"
onepixel_csv.parent.mkdir(parents=True, exist_ok=True)

with onepixel_csv.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["model", "pixels", "accuracy"])
    for model in ONEPIXEL_MODELS:
        for pixels, acc in zip(PIXELS, onepixel_accuracies[model]):
            writer.writerow([model, pixels, acc])

onepixel_csv


In [ ]:
# Plot OnePixel
plt.figure(figsize=(8, 5))
for model in ONEPIXEL_MODELS:
    plt.plot(PIXELS, onepixel_accuracies[model], marker="o", label=model)

plt.xlabel("pixels")
plt.ylabel("accuracy")
plt.title("OnePixel adversarial accuracy vs. pixels")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
